Data Cleaning 
This notebook cleans the review data by putting it to lowercase, removing special characters and numbers, removing duplicates and incomplete reviews. The file is then saved to data/intermediate.

In [10]:
import json
import re
import pandas as pd
import os
import sys

# --- Project Directory Setup ---
# Get the parent directory (project root)
sys.path.append(os.path.abspath(".."))

from src1 import load_json_lines, save_json_lines, clean_text


In [11]:
# --- File Paths Setup ---
# Base directory (go 2 levels up from /src1/)
base_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Define input/output directories using relative paths
input_dir = os.path.join(base_dir, "data", "raw")
output_dir = os.path.join(base_dir, "data", "intermediate")

input_filename = "reviews_2021-01.json"
input_file = os.path.join(input_dir, input_filename)

cleaned_file = os.path.join(output_dir, f"cleaned_{input_filename}")
na_file = os.path.join(output_dir, "NAs.json")
duplicates_file = os.path.join(output_dir, "Duplicates.json")

In [12]:
# --- Check if input file exists ---
if not os.path.exists(input_file):
    raise FileNotFoundError(f"Error: File '{input_file}' not found.")

# --- Load Data ---
reviews = load_json_lines(input_file)
df = pd.DataFrame(reviews)

# --- DataFrame creation ---
df = pd.DataFrame(reviews)

# --- Initial Checks ---
print(f"Initial data shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"Missing values per column:\n{df.isnull().sum()}")

Initial data shape: (45490, 9)
Columns: ['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date']
Missing values per column:
review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64


In [13]:
# --- Text Cleaning ---
if 'text' in df.columns:
    df['text'] = df['text'].apply(clean_text)  # Clean the text field
    df['text'] = df['text'].str.replace(r'\s+', ' ', regex=True).str.strip()  # Normalize whitespace


In [14]:
# --- Remove Duplicates ---
duplicates = df[df.duplicated(subset=['review_id'], keep=False)]
if not duplicates.empty:
    print(f"Found {duplicates.shape[0]} duplicate reviews. Saving to '{duplicates_file}'.")
    duplicates.to_json(duplicates_file, orient="records", lines=True, force_ascii=False)
    
df = df.drop_duplicates(subset=['review_id'], keep="first")  # Remove duplicate reviews


In [15]:
# --- Remove Empty Reviews ---
df = df[df['text'].str.len() > 0]  # Remove reviews with empty text

# --- Handle Missing Values ---
nas = df[df.isnull().any(axis=1)]
if not nas.empty:
    print(f"Found {nas.shape[0]} rows with missing values. Saving them to '{na_file}'.")
    nas.to_json(na_file, orient="records", lines=True, force_ascii=False)

df = df.dropna()  # Drop rows with any missing values
print(f"Data shape after handling NAs: {df.shape}")


Data shape after handling NAs: (45490, 9)


In [16]:
# --- Save Cleaned Data ---
df.to_json(cleaned_file, orient="records", lines=True, force_ascii=False)
print(f"Cleaned data saved to '{cleaned_file}'")


Cleaned data saved to 'd:\eigene Dateien\Documents\GitHub\techlabs-data-science-yelp\data\intermediate\cleaned_reviews_2021-01.json'
